In [1]:
import numpy as np
import pandas as pd
import gcsfs
from tqdm import tqdm
tqdm.pandas()
from google.cloud import bigquery
client = bigquery.Client()
from scipy.sparse import coo_matrix

# Load data

In [13]:
query = """
    WITH titles_data AS (
        SELECT DISTINCT
            TitleDetails_title, 
            TitleType, 
            cid.content_ordinal_id,
            STRING_AGG(DISTINCT TitleDetails_longsynopsis, ' ') as TitleDetails_longsynopsis,
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView` cmv
        LEFT JOIN `res-nbcupea-dev-ds-sandbox-001.recsystem.ContentOrdinalId` cid
            ON LOWER(cmv.TitleDetails_title) = LOWER(cid.program_title)
        WHERE 
            TitleDetails_longsynopsis IS NOT NULL
            AND cid.content_ordinal_id IS NOT NULL
        GROUP BY 
            TitleDetails_title, 
            TitleType,
            cid.content_ordinal_id
        ),
    train_data AS (
        SELECT program_title, ARRAY_AGG(DISTINCT t) AS tags FROM 
        res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords,
        UNNEST(tags) t
        GROUP BY program_title
    )
        
        
    SELECT a.TitleDetails_title, a.TitleType, a.content_ordinal_id, a.TitleDetails_longsynopsis, 
        b.tags
    FROM titles_data a
    LEFT JOIN train_data b
    ON LOWER(a.TitleDetails_title) = Lower(b.program_title)
    WHERE ARRAY_LENGTH(tags) > 0
"""
df_titles = client.query(query=query, location="US").to_dataframe()
df_titles

,TitleDetails_title,TitleType,content_ordinal_id,TitleDetails_longsynopsis,tags
0,Magnificent Obsession,Movie,1244,A guilt-stricken playboy (Rock Hudson) becomes...,"[Movie, eng, Drama, spa, Romance]"
1,The Phantom of the Opera,Movie,909,An acid-scarred composer (Claude Rains) rises ...,"[eng, Independent, Horror, Music, Drama, Movie..."
2,The Woman in the Fifth,Movie,950,"In Paris, an American writer (Ethan Hawke) fal...","[Thriller, spa, Movie, eng, Drama]"
3,Gilbert,Movie,386,Comedian Gilbert Gottfried recounts his career...,"[Independent, Documentary, Movie, Comedy, eng]"
4,Johnny Carson,Programme,473,Actor Albert Brooks; actress Susan Sarandon. A...,"[Comedy, Episode, Talk, eng]"
...,...,...,...,...,...
1937,St. Patrick's Day,Programme,1397,Michael realizes what's really important in life.,"[Movie, Comedy, spa, eng]"
1938,Liar Liar,Programme,513,A boy's wish comes true that his neglectful fa...,"[Comedy, spa, Courtroom, Movie, eng, Fantasy, ..."
1939,White House Down,Movie,2396,While on a tour of the White House with his yo...,"[Action & Adventure, Thriller, teens (ages 13-..."
1940,Beyond the Sea,Programme,164,An older Bobby Darin (Kevin Spacey) tells his ...,"[Drama, Movie, spa, Biography, eng, Documentar..."


In [7]:
df_titles.loc[df_titles["TitleDetails_title"]=="The Office"]

,TitleDetails_title,TitleType,content_ordinal_id,TitleDetails_longsynopsis,tags
1457,The Office,Programme,1891,Michael tries to raise employee morale with a ...,"[eng, Holiday, Episode, teens (ages 13-14), Co..."


In [10]:
tag_path = "gs://metadata-bucket-base/tfx-metadata-dev-pipeline-output/metadata_dev_edc_base_0_0_5/Transform/transform_graph/23971/transform_fn/assets/tags"
fs = gcsfs.GCSFileSystem(project="res-nbcupea-dev-ds-sandbox-001")

with fs.open(tag_path, "r") as fid:
    tags_list = fid.read().split("\n")
tags_list = [tag for tag in tags_list if tag != ""]
tags_list

['Movie',
 'eng',
 'spa',
 'Drama',
 'Episode',
 'Action & Adventure',
 'Comedy',
 'Documentary',
 'Thriller',
 'Local',
 'Horror',
 'Romance',
 'Comedy drama',
 "Children's/Family Entertainment",
 'Crime drama',
 'Fantasy',
 'Animated',
 'Mystery',
 'Science fiction',
 'Sports',
 'Reality',
 'Western',
 'Entertainment',
 'Special',
 'Romance comedy',
 'Music',
 'Sports non-event',
 'Biography',
 'Independent',
 'Holiday',
 'Crime',
 'Military & War',
 'Religion',
 'older teens (ages 15+)',
 'Musical',
 'History drama',
 'Documentary drama',
 'History',
 'Shopping',
 'Educational',
 'kids (ages 5-9)',
 'teens (ages 13-14)',
 'Talk',
 'Travel',
 'Public Affairs',
 'Gay and Lesbian',
 'Foreign',
 'Food',
 'Animals',
 'Nature',
 'tweens (ages 10-12)',
 'News',
 'Miniseries',
 'Teens',
 'Martial arts',
 'Science & Technology',
 'Anime',
 'Courtroom',
 'Health',
 'Musical comedy',
 'Sitcom',
 'Outdoors',
 'Interview',
 'Variety',
 'Home & Garden',
 'Fashion',
 'Christmas',
 'Soccer',
 'Wedd

In [16]:
def tags_to_index(tags, tags_list):
    col = np.array([tags_list.index(t) for t in tags], dtype=int)
    data = np.ones(len(col), dtype=int)
    row = np.zeros(len(col), dtype=int)
    return coo_matrix((data, (row, col)), shape=(1, len(tags_list))).A[0]

df_titles["labels"] = df_titles["tags"].progress_apply(lambda x: tags_to_index(x, tags_list))
df_titles

100%|██████████| 1942/1942 [00:00<00:00, 15064.99it/s]


,TitleDetails_title,TitleType,content_ordinal_id,TitleDetails_longsynopsis,tags,labels
0,Half Baked,Movie,1943,"Potheads (Dave Chappelle, Guillermo Diaz, Jim ...","[Independent, Movie, Comedy, Crime, eng, older...","[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Harry Potter and the Prisoner of Azkaban,Movie,1461,The young wizard (Daniel Radcliffe) and his fr...,"[Movie, Fantasy, Action & Adventure, Children'...","[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, ..."
2,Poker After Dark,Programme,641,"From Las Vegas, Nev. Diving into the world of ...","[Sports, Sports non-event, Episode, Game Show,...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Mad About Mambo,Movie,530,A soccer player (William Ash) takes mambo less...,"[eng, Romance, Movie, Romance comedy, spa]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4,Hit by Lightning,Movie,422,A nerdy restaurant manager (Jon Cryer) unexpec...,"[Comedy, Movie, spa, Drama, Crime, Romance, eng]","[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ..."
...,...,...,...,...,...,...
1937,St. Patrick's Day,Programme,1397,Michael realizes what's really important in life.,"[Movie, Comedy, spa, eng]","[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1938,Liar Liar,Programme,513,A boy's wish comes true that his neglectful fa...,"[Comedy, spa, Courtroom, Movie, eng, Fantasy, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1939,White House Down,Movie,2396,While on a tour of the White House with his yo...,"[Action & Adventure, Thriller, teens (ages 13-...","[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1940,Beyond the Sea,Programme,164,An older Bobby Darin (Kevin Spacey) tells his ...,"[Drama, Movie, spa, Biography, eng, Documentar...","[1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


# Compute cosine simliarity and recommend top 15

In [18]:
def cosine_sim(P):
    P = P / np.sqrt(np.sum(P**2, axis=1, keepdims=True))
    cos_sim_c2c = P @ P.T
    cos_sim_c2c = np.nan_to_num(cos_sim_c2c, nan=-1)
    return cos_sim_c2c

labels = np.stack(df_titles["labels"].values, axis=0)
similarity = cosine_sim(labels)

# Slice out top 15 recommendations
score = list(np.sort(similarity, axis=1)[:, ::-1][:, 1:(15+1)])
sim_c2c_argsort = np.argsort(similarity, axis=1)[:, ::-1][:, 1:]
titles = list(np.take(df_titles["TitleDetails_title"].values, sim_c2c_argsort[:, :15]))
titles_type = list(np.take(df_titles["TitleType"].values, sim_c2c_argsort[:, :15]))
synopsis = list(np.take(df_titles["TitleDetails_longsynopsis"].values, sim_c2c_argsort[:, :15]))
content_id = list(np.take(df_titles["content_ordinal_id"].values, sim_c2c_argsort[:, :15]))
dict_list = [{"TitleDetails_title": tt, "TitleType": ttype, "TitleDetails_longsynopsis": syn, "content_ordinal_id": cid, "score": sc} \
             for tt, ttype, syn, cid, sc in zip(titles, titles_type, synopsis, content_id, score)]

df_titles["top15"] = dict_list
# Calculate a type match
df_titles["type_match"] = (np.stack(titles_type) == df_titles["TitleType"][:, None]).mean(axis=1)

df_titles

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


,TitleDetails_title,TitleType,content_ordinal_id,TitleDetails_longsynopsis,tags,labels,top15,type_match
0,Half Baked,Movie,1943,"Potheads (Dave Chappelle, Guillermo Diaz, Jim ...","[Independent, Movie, Comedy, Crime, eng, older...","[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'TitleDetails_title': ['Nurse Betty', 'People...",1.000000
1,Harry Potter and the Prisoner of Azkaban,Movie,1461,The young wizard (Daniel Radcliffe) and his fr...,"[Movie, Fantasy, Action & Adventure, Children'...","[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, ...",{'TitleDetails_title': ['Harry Potter and the ...,1.000000
2,Poker After Dark,Programme,641,"From Las Vegas, Nev. Diving into the world of ...","[Sports, Sports non-event, Episode, Game Show,...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'TitleDetails_title': ['Acceleration', 'Wrest...",0.933333
3,Mad About Mambo,Movie,530,A soccer player (William Ash) takes mambo less...,"[eng, Romance, Movie, Romance comedy, spa]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","{'TitleDetails_title': ['Tammy Tell Me True', ...",1.000000
4,Hit by Lightning,Movie,422,A nerdy restaurant manager (Jon Cryer) unexpec...,"[Comedy, Movie, spa, Drama, Crime, Romance, eng]","[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","{'TitleDetails_title': ['Hit by Lightning', 'M...",1.000000
...,...,...,...,...,...,...,...,...
1937,St. Patrick's Day,Programme,1397,Michael realizes what's really important in life.,"[Movie, Comedy, spa, eng]","[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",{'TitleDetails_title': ['Slap Shot 3: The Juni...,0.066667
1938,Liar Liar,Programme,513,A boy's wish comes true that his neglectful fa...,"[Comedy, spa, Courtroom, Movie, eng, Fantasy, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'TitleDetails_title': ['Liar Liar', 'The Wend...",0.333333
1939,White House Down,Movie,2396,While on a tour of the White House with his yo...,"[Action & Adventure, Thriller, teens (ages 13-...","[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","{'TitleDetails_title': ['Tomorrow Never Dies',...",1.000000
1940,Beyond the Sea,Programme,164,An older Bobby Darin (Kevin Spacey) tells his ...,"[Drama, Movie, spa, Biography, eng, Documentar...","[1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","{'TitleDetails_title': ['Beyond the Sea', 'Som...",0.000000


In [22]:
from IPython.display import display
def query_shows_c2c(df_titles, show_name, show_res=True):
    pdf = df_titles.loc[df_titles["TitleDetails_title"]==show_name, :]
    pdf_query = pdf[["content_ordinal_id", "TitleDetails_title", "TitleType", "TitleDetails_longsynopsis"]]
    pdf_res = pd.DataFrame(pdf[f"top15"].values[0])
    if show_res:
        print("Query")
        display(pdf_query)
        display(pdf_res)
    
    return pdf_query, pdf_res

In [24]:
important_titles = ["The Office", "30 Rock", "Punky Brewster", "Parks and Recreation", "WWE Monday Night RAW", 
            "Yellowstone", "Saturday Night Live", "Law & Order: Special Victims Unit", 
            "Mr. Mercedes", "Happy Feet Two", "Zombie Tidal Wave"]

for ti in important_titles:
    _ = query_shows_c2c(df_titles, ti)

Query


,content_ordinal_id,TitleDetails_title,TitleType,TitleDetails_longsynopsis
1457,1891,The Office,Programme,Michael tries to raise employee morale with a ...


,TitleDetails_title,TitleType,TitleDetails_longsynopsis,content_ordinal_id,score
0,Roseanne,Programme,Roseanne bets Dan and his pals they will never...,1341,1.000000
1,Will & Grace,Programme,"Best friends, Will, who is a gay lawyer, and i...",1011,1.000000
2,Cheers,Programme,Diane's scorn for Sam's choice of female compa...,228,0.942809
3,The King of Queens,Programme,Doug and Carrie feel inadequate when they disc...,880,0.935414
4,A.P. Bio,Programme,When budget cuts threaten to gut Whitlock and ...,36,0.935414
5,Brooklyn Nine-Nine,Programme,Terry takes cues from Jake while on his first ...,1299,0.935414
6,George Lopez,Programme,Angie plans a surprise birthday party for Geor...,379,0.935414
7,Grace Under Fire,Programme,Quentin (Jon Paul Steuer) runs away to see his...,1205,0.894427
8,Frasier,Programme,Frasier drives his family crazy when the death...,371,0.875000
9,30 Rock,Programme,Liz tries to boost morale by taking the show o...,17,0.875000


Query


,content_ordinal_id,TitleDetails_title,TitleType,TitleDetails_longsynopsis
1349,17,30 Rock,Programme,Liz tries to boost morale by taking the show o...


...

,content_ordinal_id,TitleDetails_title,TitleType,TitleDetails_longsynopsis
1750,2320,Happy Feet Two,Movie,"Reluctant to dance, the son of Mumble (Elijah ..."


,TitleDetails_title,TitleType,TitleDetails_longsynopsis,content_ordinal_id,score
0,The Prince of Egypt,Movie,"Saved from pharaoh's infanticide, Israelite Mo...",1430,1.000000
1,Happy Feet Two,Movie,"Reluctant to dance, the son of Mumble (Elijah ...",2320,1.000000
2,Spirit: Stallion of the Cimarron,Movie,A mustang (Matt Damon) journeys through the Am...,1431,0.942809
3,Chicken Run,Movie,A dashing rooster and the hen he loves lead an...,1414,0.942809
4,The Road to El Dorado,Movie,Two Spanish rogues find the fabled city of gol...,1435,0.942809
5,Birds of Paradise,Movie,"Doused with paint, a sparrow's colors change d...",1549,0.888889
6,The Tale of Despereaux,Movie,"In the faraway kingdom of Dor, a misfit mouse ...",1601,0.888889
7,Space Jam,Movie,Bugs Bunny recruits NBA star Michael Jordan to...,1773,0.888889
8,Monster High: Freaky Fusion,Movie,"During the Bite-Centennial, the ghouls travel ...",1117,0.888889
9,Despicable Me,Movie,As he tries to execute a fiendish plot to stea...,2316,0.888889


Query


,content_ordinal_id,TitleDetails_title,TitleType,TitleDetails_longsynopsis
537,1574,Zombie Tidal Wave,Movie,Zombies wreak bloodthirsty havoc after a tidal...


,TitleDetails_title,TitleType,TitleDetails_longsynopsis,content_ordinal_id,score
0,Vampire in Vegas,Movie,An ancient vampire (Tony Todd) searches for a ...,2104,1.000000
1,The Mummy's Tomb,Movie,An old archaeologist (Dick Foran) recalls the ...,901,1.000000
2,The Mummy's Curse,Movie,"Shipped to Louisiana, mummy Kharis (Lon Chaney...",898,1.000000
3,Hood Rat,Movie,A vengeful madman (Isaiah Washington) and his ...,431,1.000000
4,The Brides of Dracula,Movie,Van Helsing (Peter Cushing) drives a stake thr...,815,1.000000
5,The Mummy's Ghost,Movie,A 3000-year-old limping mummy (Lon Chaney) see...,899,1.000000
6,The Evil of Frankenstein,Movie,The baron (Peter Cushing) and a hypnotist (Pet...,843,1.000000
7,The Creature Walks Among Us,Movie,"Scientists (Jeff Morrow, Rex Reason) capture t...",829,1.000000
8,13/13/13,Movie,In the 13th month of the 13th year of a new mi...,5,1.000000
9,Son of Frankenstein,Movie,The baron's son (Basil Rathbone) revives his f...,753,1.000000


In [27]:
# Unioned shows
query2 = """
SELECT ANY_VALUE(program_title) AS program_title, ANY_VALUE(program_longsynopsis) AS program_longsynopsis, ARRAY_AGG(DISTINCT t) AS tags
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords`,
UNNEST(tags) t
GROUP BY LOWER(program_title)
"""
df_union = client.query(query=query2, location="US").to_dataframe()

df_union

,program_title,program_longsynopsis,tags
0,Landauer - Der Präsident,"Tras la Segunda Guerra Mundial, el ex futbolis...","[spa, Movie, Biography]"
1,La Pandilla de Pícaros Va al Japón,Un agente egoísta va con un equipo de las pequ...,"[spa, Comedy, Movie]"
2,Despistadas...La Película,La renovación social de una amiga hace que una...,"[Comedy, Movie, spa, Teens]"
3,Trimming Success,(Chinese).,"[Local, Episode, Educational, eng]"
4,Fortunas de Guerra,"Un americano cínico, con trabajo temporal, ace...","[Movie, spa, Drama]"
...,...,...,...
359155,Comet,Un cínico muchacho y una joven mujer inician u...,"[older teens (ages 15+), Independent, Movie, e..."
359156,Pasión Extrema,Un guía de turistas salva a varias personas en...,"[Episode, Action Sports, spa, Sports non-event..."
359157,En su vida: La historia de John Lenon,La carrera del legendario músico John Lennon y...,"[Movie, spa, Documentary drama]"
359158,Back to the Garden,A year after the death of an inspirational the...,"[eng, Drama, Movie]"


In [31]:
df_union[df_union["program_title"]=="Mr. Mercedes"].tags.values[0]

array(['eng', 'Drama', 'Thriller', 'Episode'], dtype=object)

In [32]:
df_union[df_union["program_title"]=="The Office"].tags.values[0]

array(['Episode', 'Holiday', 'Comedy', 'Drama', 'spa',
       'teens (ages 13-14)', 'Sitcom', 'eng'], dtype=object)

In [33]:
df_union[df_union["program_title"]=="Parks and Recreation"].tags.values[0]

array(['Drama', 'Episode', 'teens (ages 13-14)', 'Sitcom', 'Romance',
       'spa', 'eng', 'older teens (ages 15+)', 'Local', 'Holiday',
       'Comedy', 'Public Affairs'], dtype=object)

In [34]:
# Unioned shows
query3 = """
SELECT ANY_VALUE(program_title) AS program_title, ANY_VALUE(program_longsynopsis) AS program_longsynopsis, 
    ARRAY_AGG(DISTINCT t) AS tags
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.merlin_data_with_lang_and_type`,
UNNEST(tags) t
GROUP BY LOWER(program_title)
"""
df_union_full = client.query(query=query3, location="US").to_dataframe()

df_union_full

,program_title,program_longsynopsis,tags
0,NightlyNws 11-06,"""NBC Nightly News, Nov 6, 2018"", 11/06/2018: N...","[News, Episode, eng]"
1,"Sam Savage, Tenor",A faculty recital featuring songs from William...,"[eng, Local, Music, Concert, Episode]"
2,Kentucky: Place & Spirit,Summer reading program in Mercer County; genea...,"[Episode, eng, Local]"
3,Live From the White House: The Millenium Lectu...,Presented by Dr. Marcia McNutt and Dr. Neil Ty...,"[Local, Science & Technology, Episode, eng, Talk]"
4,Journey Into Darkness,A blow to the head during a robbery leaves a m...,"[Horror, eng, Drama, Movie]"
...,...,...,...
410181,Nunca digas su nombre,Cuando tres estudiantes universitarios se muda...,"[Movie, spa, Horror, Thriller]"
410182,Good Morning Kentucky,(Live),"[Episode, Talk, eng, Local]"
410183,General Orders No. 9,El cineasta obert Persons llora la pérdida de ...,"[eng, Documentary, spa, Movie]"
410184,Fifteen Minutes,Un detective (Robert De Niro) de homicidios y ...,"[Action & Adventure, not for kids, spa, eng, M..."


In [46]:
df_union_full[df_union_full["program_title"]=="Yellowstone"].tags.values[0]

array(['Animals', 'eng', 'spa', 'older teens (ages 15+)', 'Movie',
       'Drama', 'Crime', 'Documentary', 'Nature', 'Mystery', 'Episode'],
      dtype=object)

In [45]:
query4 = """
SELECT TitleDetails_title, ANY_VALUE(TitleDetails_LongSynopsis) AS synopsis, 
    STRING_AGG(TitleSubgenres, ",") AS subgenres, 
    STRING_AGG(TitleTags, ",") AS title_tags
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView`
WHERE TitleDetails_title = "Mr. Mercedes"
GROUP BY TitleDetails_title
"""
df_merc = client.query(query=query4, location="US").to_dataframe()
df_merc

,TitleDetails_title,synopsis,subgenres,title_tags
0,Mr. Mercedes,Ida is hurt as Hodges and Janey grow closer; L...,"Thriller,Thriller,Thriller,Drama, Thriller,Thr...","TV, Drama, Thriller,TV, older teens (ages 15+)..."


In [42]:
set([ss.strip() for ss in df_merc.subgenres.values[0].split(",")])

{'Drama', 'Thriller'}

In [43]:
set([ss.strip() for ss in df_merc.title_tags.values[0].split(",")])

{'Drama', 'TV', 'Thriller', 'older teens (ages 15+)'}

In [ ]:
query5 = """
SELECT program_title, ANY_VALUE(TitleDetails_LongSynopsis) AS synopsis, 
    STRING_AGG(TitleSubgenres, ",") AS subgenres, 
    STRING_AGG(TitleTags, ",") AS title_tags
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView`
WHERE TitleDetails_title = "Mr. Mercedes"
GROUP BY TitleDetails_title
"""
df_merc = client.query(query=query5, location="US").to_dataframe()
df_merc

# Try to select tag labels based on semantic clustering

In [69]:
query = """
SELECT DISTINCT k AS labels
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords`,
UNNEST(keywords) k

UNION DISTINCT
SELECT DISTINCT t AS labels
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords`,
UNNEST(tags) t
"""

df_words = client.query(query=query, location="US").to_dataframe()
df_words



,labels
0,Independent
1,Horror
2,Martial Arts
3,Superhuman abilities
4,Friend
...,...
5004,Documentary drama
5005,kids (ages 5-9)
5006,Quebec Production
5007,Amazon Original


In [70]:
special_labels = ["eng", "spa", "Episode", "Movie", 'older teens (ages 15+)',
 'tweens (ages 10-12)',
 'teens (ages 13-14)',
 'big kids (ages 8-9)',
 'little kids (ages 5-7)',
 'preschoolers (ages 2-4)',
 'kids (ages 5-9)', 'not for kids']

df_words = df_words.loc[~df_words["labels"].isin(special_labels)]
df_words

,labels
0,Independent
1,Horror
2,Martial Arts
3,Superhuman abilities
4,Friend
...,...
5003,History drama
5004,Documentary drama
5006,Quebec Production
5007,Amazon Original


## Spacy embeddings

In [140]:
# Feeding into word2vec
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

df_words["embed"] = df_words["labels"].progress_apply(lambda x: nlp(x).vector)
df_words

100%|██████████| 4997/4997 [00:26<00:00, 187.20it/s]


,labels,embed,cluster,centroid_word,num_words
0,Glasgow,"[0.56278, 0.035106, -0.28147, -0.12403, 1.0665...",0,Animal's perspective,34
1,Lost loves,"[-0.0047835, 0.40439498, -0.0696, 0.08279, -0....",0,Animal's perspective,34
2,Determination,"[0.0047395, 0.66576, -0.24328, 0.24554, -0.651...",0,Animal's perspective,34
3,Death of spouse,"[-0.14613134, 0.17645667, 0.011319998, -0.1157...",0,Animal's perspective,34
4,Special education,"[-0.29166, 0.10547, 0.459795, 0.0942125, 0.120...",0,Animal's perspective,34
...,...,...,...,...,...
4992,British man,"[-0.22207999, 0.084344, 0.1609465, -0.279425, ...",139,Circus performer,24
4993,Young love,"[0.026082497, 0.294238, -0.36128998, -0.018249...",139,Circus performer,24
4994,Bangkok,"[0.11044, -0.010999, 0.15166, -0.36984, 0.7781...",139,Circus performer,24
4995,Cabaret,"[0.6916, 0.10619, 0.87455, -0.019706, 0.39388,...",139,Circus performer,24


### Kmeans clustering

In [141]:
from sklearn.cluster import KMeans
kmeans_clust = KMeans(n_clusters=140, random_state=42)
# Assign clusters
df_words["cluster"] = kmeans_clust.fit_predict(np.stack(df_words["embed"].values, axis=0))
df_words = df_words.sort_values("cluster").reset_index(drop=True)
# Figure out a "centeroid word"
# get the centroid
def centroid_word_func(pdf):
    values = np.stack(pdf["embed"].values, axis=0)
    centroid = values.mean(axis=0, keepdims=True)
    dist = np.sum((values - centroid)**2, axis=1)
    min_index = np.argmin(dist)
    pdf["centroid_word"] = pdf["labels"].iloc[min_index]
    pdf["num_words"] = pdf.shape[0]
    return pdf

df_words = df_words.groupby(by="cluster", as_index=False).progress_apply(centroid_word_func)
df_words

100%|██████████| 140/140 [00:00<00:00, 765.27it/s]


,labels,embed,cluster,centroid_word,num_words
0,Infidelity,"[-0.56933, 0.47952, 0.43122, -0.017169, 0.2706...",0,Cheating,4
1,Adultery,"[-0.19643, 0.11532, 0.88786, -0.3515, 0.21305,...",0,Cheating,4
2,Cheating,"[-1.2114, -0.012163, 0.012448, -0.17388, -0.35...",0,Cheating,4
3,Cheater,"[-0.21375, 0.14478, -0.40881, 0.28415, -0.2359...",0,Cheating,4
4,Hit and run,"[-0.044889998, 0.16249935, -0.11236667, 0.0569...",1,Changing the past,156
...,...,...,...,...,...
4992,Refugee,"[0.058596, 0.076589, -0.12956, 0.80858, 0.4088...",139,Prison camp,19
4993,Troubled youth,"[0.38193, 0.375385, 0.30402, 0.226885, 0.10895...",139,Prison camp,19
4994,Boot camp,"[0.014685005, 0.117116496, 0.20905, -0.1385050...",139,Prison camp,19
4995,Youth,"[0.49517, 0.57949, 0.42464, 0.10126, 0.33488, ...",139,Prison camp,19


## Google hub embeddings

In [4]:
# This results bullshit embeddings

import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

TFHUB_HANDLE_PREPROCESSOR = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
TFHUB_HANDLE_ENCODER = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"


text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='synopsis')
preprocessing_layer = hub.KerasLayer(TFHUB_HANDLE_PREPROCESSOR, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(TFHUB_HANDLE_ENCODER, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
model = tf.keras.Model(text_input, net)

In [71]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

model = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")

In [72]:
dataset = tf.data.Dataset.from_tensor_slices(df_words["labels"].values).batch(50)
res = []
for batch in tqdm(dataset):
     res.append(model(batch))
df_words["embed"] = list(tf.concat(res, axis=0).numpy())
df_words.to_pickle("./scratch/hub_embed_keywords_embed.pkl")
df_words

100%|██████████| 100/100 [00:00<00:00, 722.70it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,labels,embed
0,Independent,"[0.011223246, 0.12829885, 0.06841187, -0.08473..."
1,Horror,"[-0.109493434, 0.084715165, -0.021021072, 0.10..."
2,Martial Arts,"[-0.04248964, 0.20790195, -0.10532143, 0.03185..."
3,Superhuman abilities,"[0.05927546, 0.01962894, 0.027596451, -0.02112..."
4,Friend,"[-0.06137415, 0.006071199, 0.0049916552, -0.12..."
...,...,...
5003,History drama,"[-0.030099658, 0.06847251, -0.09140941, -0.039..."
5004,Documentary drama,"[-0.03812892, 0.13716692, -0.064767584, -0.065..."
5006,Quebec Production,"[-0.06971015, -0.052805364, 0.076412186, 0.047..."
5007,Amazon Original,"[-0.02408965, -0.0029515543, 0.16048776, 0.086..."


In [73]:
from sklearn.cluster import KMeans
kmeans_clust = KMeans(n_clusters=140, random_state=42)
# Assign clusters
df_words["cluster"] = kmeans_clust.fit_predict(np.stack(df_words["embed"].values, axis=0))
df_words = df_words.sort_values("cluster").reset_index(drop=True)
# Figure out a "centeroid word"
# get the centroid
def centroid_word_func(pdf):
    values = np.stack(pdf["embed"].values, axis=0)
    centroid = values.mean(axis=0, keepdims=True)
    dist = np.sum((values - centroid)**2, axis=1)
    min_index = np.argmin(dist)
    pdf["centroid_word"] = pdf["labels"].iloc[min_index]
    pdf["num_words"] = pdf.shape[0]
    return pdf

df_words = df_words.groupby(by="cluster", as_index=False).progress_apply(centroid_word_func)
df_words

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
100%|██████████| 140/140 [00:00<00:00, 548.63it/s]


,labels,embed,cluster,centroid_word,num_words
0,Brother relationship,"[-0.013943148, 0.11200457, 0.03402433, -0.0021...",0,Interracial relationship,20
1,Cousin relationship,"[-0.005889324, 0.034345884, 0.0034403205, -0.0...",0,Interracial relationship,20
2,Unusual friendship,"[0.004125057, 0.031462085, -0.035512228, 0.026...",0,Interracial relationship,20
3,Engagement,"[-0.023923595, -0.03137951, -0.05741388, 0.000...",0,Interracial relationship,20
4,Unlikely friendship,"[-0.035018355, 0.09890234, -0.012741492, 0.095...",0,Interracial relationship,20
...,...,...,...,...,...
4992,Fighter,"[-0.05542894, 0.053740177, -0.034873147, -0.02...",139,Fighter,36
4993,Batman,"[-0.10051098, 0.057055548, -0.11739935, 0.1009...",139,Fighter,36
4994,Grudge,"[-0.12620774, -0.02534555, 0.031746693, 0.0180...",139,Fighter,36
4995,Vendetta,"[-0.10627611, -0.056765504, -0.1088888, 0.0624...",139,Fighter,36


In [91]:
df_words.loc[df_words["cluster"]==28, :]

,labels,embed,cluster,centroid_word,num_words
1081,Hispanic culture,"[-0.037192516, 0.18877167, 0.19013783, -0.0742...",28,American culture,22
1082,Middle Ages,"[-0.016006334, 0.032668363, 0.040764175, 0.051...",28,American culture,22
1083,Black culture,"[-0.036916267, 0.047661245, 0.09371184, -0.097...",28,American culture,22
1084,Pop culture,"[-0.07359134, 0.14295755, 0.09240613, -0.01135...",28,American culture,22
1085,Teen culture,"[-0.08005376, 0.24407361, 0.15350223, -0.05606...",28,American culture,22
1086,American culture,"[-0.0072190096, 0.06315037, 0.0977883, -0.1013...",28,American culture,22
1087,Corporate culture,"[-0.042208716, 0.17224924, 0.10305093, -0.0843...",28,American culture,22
1088,Old age,"[0.094340935, 0.016945757, 0.13947445, 0.02418...",28,American culture,22
1089,Religious beliefs,"[-0.0013960616, 0.21556112, -0.009749432, -0.1...",28,American culture,22
1090,Western myths,"[-0.008615203, -0.0520459, 0.07878148, -0.0586...",28,American culture,22


In [92]:
# Adding the special words
df_special = pd.DataFrame({"labels": [kk for kk in special_labels], 
                           "embed" : [[]] * len(special_labels),
                           "cluster": np.arange(140, 140+len(special_labels)),
                          "centroid_word": [kk for kk in special_labels],
                          "num_words": np.ones(len(special_labels), dtype=int)}
                         )
df_final_map = df_words.append(df_special)
df_final_map

,labels,embed,cluster,centroid_word,num_words
0,Brother relationship,"[-0.013943148, 0.11200457, 0.03402433, -0.0021...",0,Interracial relationship,20
1,Cousin relationship,"[-0.005889324, 0.034345884, 0.0034403205, -0.0...",0,Interracial relationship,20
2,Unusual friendship,"[0.004125057, 0.031462085, -0.035512228, 0.026...",0,Interracial relationship,20
3,Engagement,"[-0.023923595, -0.03137951, -0.05741388, 0.000...",0,Interracial relationship,20
4,Unlikely friendship,"[-0.035018355, 0.09890234, -0.012741492, 0.095...",0,Interracial relationship,20
...,...,...,...,...,...
7,big kids (ages 8-9),[],147,big kids (ages 8-9),1
8,little kids (ages 5-7),[],148,little kids (ages 5-7),1
9,preschoolers (ages 2-4),[],149,preschoolers (ages 2-4),1
10,kids (ages 5-9),[],150,kids (ages 5-9),1


In [93]:
df_final_map.to_gbq("metadata_enhancement.keyword_clusters",
    project_id=client.project,
    if_exists='replace')

1it [00:10, 10.41s/it]


# Mapping the keywords and tags to the centroid words

In [113]:
%%bigquery temp
SELECT * 
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.merlin_data_with_lang_type_keywords`
WHERE program_title = "Yellowstone"

Downloading: 100%|██████████| 4/4 [00:01<00:00,  2.62rows/s]


In [114]:
temp

,program_title,program_type,program_longsynopsis,tags,keyword_length,keywords
0,Yellowstone,Movie,Soar over majestic Yellowstone National Park a...,"[Movie, Documentary, eng]",69,"[News, Gangster, Special, Estrangement, 2010s,..."
1,Yellowstone,Movie,El cineasta Kieth Merrill muestra la belleza d...,"[spa, Movie, Documentary]",69,"[Drama, Gutsy, Love, Ranchers, Cowboys, Yellow..."
2,Yellowstone,Movie,Crooks die looking for their loot around hot g...,"[Drama, eng, Crime, Mystery, Movie]",69,"[National parks, Employer, 1930s, Pursuit, Mur..."
3,Yellowstone,Episode,Enemies surround John Dutton at every turn. Jo...,"[spa, older teens (ages 15+), Animals, Episode...",69,"[Adventure, Reservation life, Gangsters, Beaut..."


In [104]:
query = """
    WITH orig AS (
        SELECT program_title, program_type, program_longsynopsis, tags
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords`
        
        UNION ALL
        
        SELECT program_title, program_type, program_longsynopsis, keywords AS tags
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords`
    ),
    
    
    mapper AS (
        SELECT labels, centroid_word, cluster
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.keyword_clusters`
    ),
    
    collections AS (
        SELECT a.program_title, a.program_type, a.program_longsynopsis, 
        ARRAY_AGG(DISTINCT b.centroid_word) AS tags, ARRAY_AGG(DISTINCT b.cluster) AS cluster
        FROM orig a
        JOIN mapper b
        ON b.labels IN UNNEST(a.tags)
        GROUP BY a.program_title, a.program_type, a.program_longsynopsis
    )
    
    SELECT program_title, program_type, program_longsynopsis, tags
    FROM collections
"""

table_id = f"{client.project}.metadata_enhancement.synopsis_151_clustered_tags"
job_config = bigquery.QueryJobConfig(destination=table_id, write_disposition="WRITE_TRUNCATE")

df_train = client.query(query=query, job_config=job_config,  location="US").to_dataframe()
df_train


,program_title,program_type,program_longsynopsis,tags
0,El Bus del Terror,Movie,Un niño aspira con ser escritor de grande y de...,"[Bus, Man, Expedition, Child, Necromancer, Spi..."
1,War Pigs,Movie,"Un capitán del ejército estadounidense, ayudad...","[Bus, War, Syria, Movie, Confusion, Spirited, ..."
2,Indiscretion of an American Wife,Movie,A woman (Anne Archer) is torn between loyalty ...,"[Bus, eng, Heart, Thief, Immigrant, Daughter, ..."
3,Schizo,Movie,Someone wants a woman (Lynne Frederick) to thi...,"[Bus, eng, Movie, Extortion, Death, Superhero,..."
4,Los intocables,Movie,Eliot Ness (Kevin Costner) and his men fight A...,"[Bus, eng, Superstar, Confusion, Chicago, Bad ..."
...,...,...,...,...
514338,La Bella y la Bestia,Episode,Cat confronta a su padre biológico acerca de s...,"[Father/child relationship, Childcare, Owner, ..."
514339,18 Kids and Counting,Episode,How the Duggar children are schooled at home; ...,"[Father/child relationship, Childcare, Church,..."
514340,Leave No Trace,Episode,David Wilson relates how forensic science has ...,"[Father/child relationship, Expedition, Daught..."
514341,It's Always Sunny in Philadelphia,Episode,Frank inventa un plan para socavar a los basur...,"[Father/child relationship, Girlfriend, eng, R..."


In [110]:
df_train.loc[df_train["program_title"]=="The Office"].program_longsynopsis.values[0].split(" ")[:256]

['Pam',
 'está',
 'avergonzada',
 'cuando',
 'nadie',
 'más',
 'en',
 'la',
 'compañía',
 'usa',
 'un',
 'disfraz',
 'en',
 'Halloween;',
 'Holly',
 'y',
 'Michael',
 'reciben',
 'noticias',
 'espantosas',
 'de',
 'la',
 'compañía;',
 'Dwight',
 'trata',
 'de',
 'comprender',
 'a',
 'Andy.',
 'Strife',
 'on',
 'the',
 'party-planning',
 'committee',
 'results',
 'in',
 'two',
 'competing',
 'Christmas',
 'parties;',
 'Michael',
 'gets',
 'dumped',
 'for',
 'the',
 'holidays.',
 'Michael',
 'comes',
 'up',
 'with',
 'an',
 'idea',
 'that',
 'causes',
 'a',
 'huge',
 'problem',
 'in',
 'the',
 'office;',
 'Pam,',
 'Jim',
 'and',
 'Andy',
 'each',
 'give',
 'Kevin',
 'different',
 'advice',
 'on',
 'wooing',
 'a',
 'woman.',
 'David',
 'learns',
 'his',
 'Slough',
 'branch',
 'is',
 'to',
 'be',
 'merged',
 'with',
 'another.',
 'Andy',
 'decide',
 'dejar',
 'su',
 'trabajo',
 'y',
 'perseguir',
 'su',
 'sueño;',
 'Dwight',
 'tiene',
 'el',
 'mejor',
 'día',
 'en',
 'su',
 'trabajo;',
 'J

In [9]:
query = """
    WITH orig AS (
        SELECT program_title, program_type, program_longsynopsis, tags
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords`
        
        UNION ALL
        
        SELECT program_title, program_type, program_longsynopsis, keywords AS tags
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords`
    ),
    
    unionized AS (
        SELECT ANY_VALUE(program_title) AS program_title, 
            ANY_VALUE(program_type) AS program_type, 
            ANY_VALUE(program_longsynopsis) AS program_longsynopsis, 
            ARRAY_AGG(DISTINCT t) AS tags
        FROM orig,
        UNNEST(tags) t
        GROUP BY LOWER(program_title)
    ),
    
    mapper AS (
        SELECT labels, centroid_word, cluster
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.keyword_clusters`
    ),
    
    collections AS (
        SELECT a.program_title, a.program_type, a.program_longsynopsis, 
        ARRAY_AGG(b.centroid_word) AS tags, ARRAY_AGG(DISTINCT b.cluster) AS cluster
        FROM unionized a
        JOIN mapper b
        ON b.labels IN UNNEST(a.tags)
        GROUP BY a.program_title, a.program_type, a.program_longsynopsis
    )
    
    SELECT program_title, program_type, program_longsynopsis, cluster,
        ARRAY(
            SELECT AS STRUCT tags, COUNT(1) `count`
            FROM collections.tags tags
            GROUP BY tags
        ) stats
    FROM collections
"""

table_id = f"{client.project}.metadata_enhancement.synopsis_151_clustered_tags"
job_config = bigquery.QueryJobConfig(destination=table_id, write_disposition="WRITE_TRUNCATE")

df_train = client.query(query=query,  location="US").to_dataframe()
df_train

,program_title,program_type,program_longsynopsis,cluster,stats
0,British Formula 3,Episode,Desde Croft Circuit. Destacados del más presti...,"[142, 15, 22, 141, 66, 50]","[{'tags': 'Episode', 'count': 1}, {'tags': 'Us..."
1,Inquiries on CPAC,Episode,Quebec Premier Jean Charest speaks to the Inqu...,"[142, 27, 37]","[{'tags': 'Politics & Government', 'count': 1}..."
2,The Stars Caravan,Movie,A projectionist travels across Kyrgyzstan to b...,"[143, 5, 27, 92]","[{'tags': 'Old West myths', 'count': 2}, {'tag..."
3,Buscándolo vivo o muerto,Movie,Un ranchero de Arizona y su hijo se unen para ...,"[143, 136, 141]","[{'tags': 'Movie', 'count': 1}, {'tags': 'spa'..."
4,Inside the Refugee Crisis,Episode,The Sister's perspective on the humanitarian r...,"[142, 22, 37, 27]","[{'tags': 'Politics & Government', 'count': 1}..."
...,...,...,...,...,...
359155,Fall of the Shogun's Militia,Movie,Famed mercenaries rise and fall.,"[143, 27, 98]","[{'tags': 'Movie', 'count': 1}, {'tags': 'Musi..."
359156,Star Wars: Episodio I - La amenaza fantasma,Movie,Cuando las fuerzas enemigas amenazan a la pací...,"[9, 44, 107, 143, 82, 19, 105, 118, 48, 74, 89...","[{'tags': 'Sports and Martial Arts', 'count': ..."
359157,Estamos Muertos O Qué?,Movie,Dos policías de Los Ángeles y una mujer de rel...,"[72, 55, 99, 67, 16, 130, 40, 26, 44, 30, 75, ...","[{'tags': 'Film set', 'count': 1}, {'tags': 'A..."
359158,Hunting Evil,Movie,A broken man meets an enigmatic entity that pr...,"[22, 98, 27, 143, 92]","[{'tags': 'Movie', 'count': 1}, {'tags': 'Film..."


In [16]:
df_train.loc[df_train["program_title"]=="The Office"].stats.values

array([array([{'tags': 'Personal assistant', 'count': 2},
       {'tags': 'Norfolk', 'count': 1},
       {'tags': 'Unhappy marriage', 'count': 2},
       {'tags': 'Love at first sight', 'count': 2},
       {'tags': 'teens (ages 13-14)', 'count': 1},
       {'tags': 'Time travel', 'count': 1},
       {'tags': 'Musical comedy', 'count': 3},
       {'tags': 'United States presidency', 'count': 1},
       {'tags': 'Amusing', 'count': 5},
       {'tags': 'Crooked cop', 'count': 1},
       {'tags': 'Politics & Government', 'count': 1},
       {'tags': 'Shopkeeper', 'count': 1},
       {'tags': 'Care-a-lot', 'count': 1}, {'tags': 'spa', 'count': 1},
       {'tags': '1970s', 'count': 2},
       {'tags': 'Failing business', 'count': 1},
       {'tags': 'Episode', 'count': 1},
       {'tags': 'Home ownership', 'count': 1},
       {'tags': 'Howard the Duck', 'count': 1},
       {'tags': 'Hooterville', 'count': 1}], dtype=object)], dtype=object)

In [20]:
df_titles.loc[df_titles["TitleDetails_title"]=="The Office"].tags.values

array([array(['eng', 'Holiday', 'Episode', 'teens (ages 13-14)', 'Comedy', 'spa',
       'Sitcom', 'Drama'], dtype=object)], dtype=object)

In [24]:
dfff = _

In [32]:
dfff.tags.values[0]

array(['spa', 'Friendship', 'Office', 'Secretary', 'Co-worker', 'Drama',
       'eng', '2010s', 'Boss', 'Comedy', 'Cheeky', 'Manager', 'Work',
       'Hilarious', 'Holiday', 'Employee', 'Relationships', 'Witty',
       'Episode', 'Workplace politics', 'Accountant', 'Salesman',
       'Sitcom', 'Love interest', 'teens (ages 13-14)', 'Endearing',
       'Amusing', 'England', '2000s', 'Dunder Mifflin'], dtype=object)

# Inspect the labels

In [2]:
query = """
    SELECT * FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_151_clustered_tags`
    WHERE program_title IN ("Yellowstone", "Mr. Mercedes", "Parks and Recreation", "The Office", 
    "Punky Brewster", "Treadstone")
"""
df_sub = client.query(query=query, location="US").to_dataframe()

df_sub

,program_title,program_type,program_longsynopsis,tags
0,Punky Brewster,Episode,Punky y Cherie hacen de casamenteras haciendo ...,"[Chicago, Amusing, Journalist, Unhappy marriag..."
1,Mr. Mercedes,Episode,When a visit from Lou elicits an unprecedented...,"[Unusual behavior, Film set, Personal vendetta..."
2,Parks and Recreation,Episode,Leslie passes a city-wide tax on huge sodas in...,"[Amusing, Aspiring superhero, teens (ages 13-1..."
3,The Office,Episode,Pam está avergonzada cuando nadie más en la co...,"[Episode, spa, United States presidency, Home ..."
4,Yellowstone,Movie,El cineasta Kieth Merrill muestra la belleza d...,"[Arizona, Shopkeeper, Television star, Unusual..."
